<a href="https://colab.research.google.com/github/aaabhijith13/linkedIN_posts/blob/main/SparkMLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#SPARK MLlib

In [29]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import col, count, when, isnull, skewness, log1p, col

In [9]:
TARGET = "median_house_value"
spark = SparkSession.builder.getOrCreate()

In [10]:
df = spark.read.csv("sample_data/california_housing_train.csv", header=True, inferSchema=True) #local files, we have header = True because first row is the header in the csv


In [16]:
df.printSchema()
df.describe().show()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)

+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|          latitude|housing_median_age|      total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|
+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              17000|             17000|          

In [21]:
df.select([count(when(col(c).isNull(), c)).alias(c)for c in df.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|        0|       0|                 0|          0|             0|         0|         0|            0|                 0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+



In [22]:
housing_df = df.withColumn(
    "rooms_per_household",
    col("total_rooms") / col("households")
).withColumn(
    "bedrooms_per_room",
    col("total_bedrooms") / col("total_rooms")
).withColumn(
    "population_per_household",
    col("population") / col("households")
)


In [23]:
FEATURES = [col for col in housing_df.columns if col != TARGET]
FEATURES #Median house value is a continuous variable → regression problem.

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'rooms_per_household',
 'bedrooms_per_room',
 'population_per_household']

In [30]:
housing_df.select(
    skewness("housing_median_age").alias("housing_median_age"),
    skewness("population").alias("population_skew"),
    skewness("total_rooms").alias("rooms_skew"),
    skewness("total_bedrooms").alias("bedrooms_skew"),
    skewness("households").alias("households_skew"),
    skewness("median_income").alias("median_income_skew"),
    skewness("rooms_per_household").alias("rooms_per_household_skew"),
    skewness("bedrooms_per_room").alias("bedrooms_per_room_skew"),
    skewness("population_per_household").alias("population_per_household_skew"),

).show()

+-------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------------+----------------------+-----------------------------+
| housing_median_age|  population_skew|       rooms_skew|    bedrooms_skew|  households_skew|median_income_skew|rooms_per_household_skew|bedrooms_per_room_skew|population_per_household_skew|
+-------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------------+----------------------+-----------------------------+
|0.06488830685009192|5.186754171863749|4.002376807943188|3.322343534496116|3.342373413978119|1.6265495626993831|      21.790335077445405|    2.2354174874913135|           112.23542370003474|
+-------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------------+----------------------+-----------------------------+



In [ ]:
log_cols = ["population","total_rooms","total_bedrooms","households",
            "rooms_per_household",
"population_per_household",
    "median_income",
    "bedrooms_per_room"
]

for c in log_cols:
    housing_df = housing_df.withColumn(f"log_{c}", log1p(col(c)))